In [2]:
import pandas as pd
import numpy as np
import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
import matplotlib.cm as cm # Matplotlib and associated plotting modules
import matplotlib.colors as colors
from sklearn.cluster import KMeans # import k-means from clustering stage
from sklearn.metrics import silhouette_score
import folium # map rendering library
from shapely.geometry import Polygon
print('Libraries imported.')

Libraries imported.


In [67]:
file_suburb_pcode = 'victoria_postcodes.csv'
data_suburb_pcode = pd.read_csv(file_suburb_pcode, names=['Postcode','Suburb','Region'])
data_suburb_pcode['Suburb'] = data_suburb_pcode['Suburb'].str.lower()
data_suburb_pcode['Region'] = data_suburb_pcode['Region'].str.lower()
data_suburb_pcode = data_suburb_pcode.loc[data_suburb_pcode['Region']!='vic far country']
data_suburb_pcode = data_suburb_pcode.loc[data_suburb_pcode['Region']!='vic country']
data_suburb_pcode.head(10)


,Postcode,Suburb,Region
0,3000,melbourne,melbourne city
1,3001,melbourne,melbourne city
2,3002,east melbourne,melbourne city
3,3003,west melbourne,melbourne city
4,3004,melbourne,melbourne city
5,3006,south wharf,moorabbin
6,3006,southbank,moorabbin
7,3008,docklands,melbourne city
8,3010,university of melbourne,melbourne city
9,3011,footscray,footscray


In [46]:
file_suburb_coord = 'victoria_suburb_geocoordiantes.json'
with open(file_suburb_coord, 'r') as myfile:
    data_suburb_coord = myfile.read()

data_suburb_coord = pd.read_json(data_suburb_coord)
data_suburb_coord['geometry'][0]['coordinates']


,geometry,type,properties
0,"{'type': 'Polygon', 'coordinates': [[[145.0012...",Feature,"{'SSC_NAME': 'Abbotsford (Vic.)', 'State': 'Vi..."
1,"{'type': 'Polygon', 'coordinates': [[[144.8894...",Feature,"{'SSC_NAME': 'Aberfeldie', 'State': 'Victoria'..."
2,"{'type': 'Polygon', 'coordinates': [[[145.6416...",Feature,"{'SSC_NAME': 'Acheron', 'State': 'Victoria', '..."
3,"{'type': 'Polygon', 'coordinates': [[[144.1021...",Feature,"{'SSC_NAME': 'Aireys Inlet', 'State': 'Victori..."
4,"{'type': 'Polygon', 'coordinates': [[[144.8894...",Feature,"{'SSC_NAME': 'Airport West', 'State': 'Victori..."
5,"{'type': 'Polygon', 'coordinates': [[[144.7569...",Feature,"{'SSC_NAME': 'Albanvale', 'State': 'Victoria',..."
6,"{'type': 'Polygon', 'coordinates': [[[144.9701...",Feature,"{'SSC_NAME': 'Albert Park (Vic.)', 'State': 'V..."
7,"{'type': 'Polygon', 'coordinates': [[[146.6472...",Feature,"{'SSC_NAME': 'Alberton (Vic.)', 'State': 'Vict..."
8,"{'type': 'Polygon', 'coordinates': [[[144.8153...",Feature,"{'SSC_NAME': 'Albion (Vic.)', 'State': 'Victor..."
9,"{'type': 'Polygon', 'coordinates': [[[145.8805...",Feature,"{'SSC_NAME': 'Alexandra (Vic.)', 'State': 'Vic..."


In [65]:
file_age_by_pcode = 'age_by_postcode.csv'
data_age_by_pcode = pd.read_csv(file_age_by_pcode)
data_age_by_pcode.head(10)

,Postcode,0-4 years,5-9 years,10-14 years,15-19 years,20-24 years,25-29 years,30-34 years,35-39 years,40-44 years,...,60-64 years,65-69 years,70-74 years,75-79 years,80-84 years,85-89 years,90-94 years,95-99 years,100 years and over,Total
0,3000,735,320,161,3561,11898,8697,5045,2325,1246,...,572,448,256,127,73,42,31,6,0,37979
1,3002,157,83,60,86,343,725,680,444,274,...,365,301,242,105,77,93,40,17,6,4957
2,3003,184,115,97,193,915,1252,933,488,355,...,112,91,74,45,12,13,4,0,0,5516
3,3004,284,177,106,257,819,1432,1365,907,600,...,482,507,349,214,124,131,70,41,8,9311
4,3005,32,16,6,14,56,90,96,62,29,...,11,16,6,7,0,0,0,0,0,523
5,3006,584,206,118,814,3211,4165,3436,1757,975,...,489,454,255,149,66,40,8,3,5,18811
6,3008,524,135,81,427,1513,2281,1696,891,561,...,333,289,139,61,32,11,7,0,0,10438
7,3010,3,0,6,756,658,80,28,14,4,...,11,7,10,0,0,0,0,0,0,1595
8,3011,1348,827,511,825,2287,3183,2847,2134,1465,...,723,584,385,381,295,242,112,34,5,21464
9,3012,1987,1470,996,1071,2118,3035,3093,2539,2061,...,997,734,532,482,423,288,141,33,4,26295


In [66]:
file_income_by_pcode = 'income_by_postcode.csv'
data_income_by_pcode = pd.read_csv(file_income_by_pcode)
data_income_by_pcode.head(10)

,Postcode,Negative income,Nil income,"1-149 (1-7,799)","150-299 (7,800-15,599)","300-399 (15,600-20,799)","400-499 (20,800-25,999)","500-649 (26,000-33,799)","650-799 (33,800-41,599)","800-999 (41,600-51,999)","1,000-1,249 (52,000-64,999)","1,250-1,499 (65,000-77,999)","1,500-1,749 (78,000-90,999)","1,750-1,999 (91,000-103,999)","2,000-2,999 (104,000-155,999)","3,000 or more (156,000 or more)",Not stated,Not applicable,Total
0,3000,379,9718,1259,1878,2162,2051,2168,1913,2100,2231,1618,1321,890,1475,925,4687,1214,37979
1,3002,23,231,96,154,125,168,198,209,296,383,382,372,272,529,615,607,299,4957
2,3003,36,586,150,242,252,247,258,273,336,456,331,310,202,351,223,865,395,5516
3,3004,32,777,170,248,263,315,333,405,608,798,698,650,485,935,787,1234,563,9311
4,3005,5,33,0,25,14,18,21,23,22,25,25,29,20,42,36,135,49,523
5,3006,103,2404,386,585,644,690,799,914,1342,1625,1285,1099,856,1584,1145,2454,904,18811
6,3008,86,1574,217,322,347,313,408,450,622,791,736,640,456,867,636,1232,743,10438
7,3010,8,444,422,318,107,44,48,19,14,13,6,11,5,26,16,80,13,1595
8,3011,94,1685,655,1478,1518,1364,1278,1195,1498,1608,1218,960,661,1094,409,2074,2684,21464
9,3012,110,2109,728,1621,1739,1651,1425,1549,1910,1919,1416,1186,816,1169,481,2013,4453,26295


In [69]:
file_suburb = 'greater_melbourne_suburb_data.csv'
data_suburb = pd.read_csv(file_suburb)
print(data_suburb.shape)
data_suburb.head(10)


(528, 5)


,Suburb,Postcode,Longitude,Latitude,Area
0,Abbotsford,3067,144.999797,-37.804704,2.223370
1,Aberfeldie,3040,144.897425,-37.759636,1.971027
2,Airport West,3042,144.881337,-37.723862,4.653908
3,Albanvale,3021,144.768545,-37.746106,2.416946
4,Albert Park,3206,144.963193,-37.844753,4.345716
5,Albion,3020,144.816046,-37.776356,3.245401
6,Alphington,3078,145.030875,-37.780379,3.549580
7,Altona,3018,144.813054,-37.860082,20.914229
8,Altona Meadows,3028,144.784470,-37.880983,13.368719
9,Altona North,3025,144.834670,-37.837965,17.778935


In [70]:
address = 'Melbourne, VIC'

geolocator = Nominatim(user_agent="explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Melbourne are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Melbourne are -37.8142176, 144.9631608.


In [73]:
# create map of Melbourne using latitude and longitude values
map_melbourne_metro = folium.Map(location=[latitude, longitude], zoom_start=9)

# add markers to map
for lat, lng, suburb in zip(data_suburb['Latitude'], data_suburb['Longitude'], data_suburb['Suburb']):
    label = '{}, VIC'.format(suburb)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_melbourne_metro)  
    
map_melbourne_metro

In [19]:
# @hidden_cell
CLIENT_ID = 'NTTIKY2SV2TRCOLPLUEFE1WVADLWQN1IFGFTPUIDRN5OG0JQ' # your Foursquare ID
CLIENT_SECRET = 'HZSISWO4QRNIQBIQI2KMHKN5FN12YNYR2DXRZINQPK2VOZH1' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100

In [20]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(name, 
                             lat, 
                             lng, 
                             v['venue']['name'], 
                             v['venue']['location']['lat'], 
                             v['venue']['location']['lng'],  
                             v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Suburb', 
                             'Suburb Latitude', 
                             'Suburb Longitude', 
                             'Venue', 
                             'Venue Latitude', 
                             'Venue Longitude', 
                             'Venue Category']
    
    return(nearby_venues)

In [22]:
melbourne_venues = getNearbyVenues(names = data_suburb['Suburb'],
                                   latitudes = data_suburb['Latitude'],
                                   longitudes = data_suburb['Longitude'])
print(melbourne_venues.shape)
melbourne_venues.head(10)

Abbotsford
Aberfeldie
Airport West
Albanvale
Albert Park
Albion
Alphington
Altona
Altona Meadows
Altona North
Ardeer
Armadale
Arthurs Creek
Arthurs Seat
Ascot Vale
Ashbourne
Ashburton
Ashwood
Aspendale
Aspendale Gardens
Attwood
Avondale Heights
Avonsleigh
Bacchus Marsh
Badger Creek
Balaclava
Balliang
Balnarring
Balnarring Beach
Balwyn
Balwyn North
Bangholme
Baxter
Bayles
Bayswater
Bayswater North
Beaconsfield
Beaconsfield Upper
Beaumaris
Belgrave
Belgrave Heights
Belgrave South
Bellfield
Bend Of Islands
Bentleigh
Bentleigh East
Berwick
Beveridge
Bittern
Black Rock
Blackburn
Blackburn North
Blackburn South
Blairgowrie
Blind Bight
Bonbeach
Boneo
Boronia
Botanic Ridge
Box Hill
Box Hill North
Box Hill South
Braeside
Braybrook
Briar Hill
Brighton
Brighton East
Broadmeadows
Brookfield
Brooklyn
Brunswick
Brunswick East
Brunswick West
Bulla
Bulleen
Bullengarook
Bundoora
Burnley
Burnside
Burnside Heights
Burwood
Burwood East
Cairnlea
Calder Park
Camberwell
Campbellfield
Cannons Creek
Canterbury

,Suburb,Suburb Latitude,Suburb Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Abbotsford,-37.804704,144.999797,Lentil As Anything,-37.802724,145.003507,Vegetarian / Vegan Restaurant
1,Abbotsford,-37.804704,144.999797,The Kitchen at Weylandts,-37.805311,144.997345,Café
2,Abbotsford,-37.804704,144.999797,Three Bags Full,-37.807318,144.996603,Café
3,Abbotsford,-37.804704,144.999797,Abbotsford Convent,-37.802574,145.004410,Cultural Center
4,Abbotsford,-37.804704,144.999797,Abbotsford Convent Gardens,-37.802454,145.003510,Garden
5,Abbotsford,-37.804704,144.999797,Slow Food Market,-37.802481,145.003597,Farmers Market
6,Abbotsford,-37.804704,144.999797,The Park Hotel,-37.802769,144.997029,Pub
7,Abbotsford,-37.804704,144.999797,Retreat Hotel,-37.801126,144.997548,Pub
8,Abbotsford,-37.804704,144.999797,Salvos Store,-37.805980,144.998260,Thrift / Vintage Store
9,Abbotsford,-37.804704,144.999797,Kappaya Japanese Soul Food,-37.802590,145.003582,Japanese Restaurant


In [28]:
# one hot encoding
melbourne_onehot = pd.get_dummies(melbourne_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
melbourne_onehot['Suburb'] = melbourne_venues['Suburb'] 

# move neighborhood column to the first column
fixed_columns = [melbourne_onehot.columns[-1]] + list(melbourne_onehot.columns[:-1])
melbourne_onehot = melbourne_onehot[fixed_columns]

# Now we see the frequency data for each type of venue in Toronto
melbourne_grouped = melbourne_onehot.groupby('Suburb').mean().reset_index()
melbourne_grouped.head(10)

,Suburb,Accessories Store,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Arts & Entertainment,...,Wedding Hall,Whisky Bar,Wine Bar,Wine Shop,Winery,Women's Store,Yoga Studio,Yunnan Restaurant,Zoo,Zoo Exhibit
0,Abbotsford,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Airport West,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Albanvale,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Albert Park,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Albion,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Alphington,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,Altona,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,Altona Meadows,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,Altona North,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,Ardeer,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [30]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Suburb']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['Suburb'] = melbourne_grouped['Suburb']

for ind in np.arange(melbourne_grouped.shape[0]):
    venues_sorted.iloc[ind, 1:] = return_most_common_venues(melbourne_grouped.iloc[ind, :], num_top_venues)

venues_sorted.head(10)

,Suburb,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Abbotsford,Café,Pub,Garden,Cultural Center,Thrift / Vintage Store,Vegetarian / Vegan Restaurant,Convenience Store,Farmers Market,Sporting Goods Shop,Japanese Restaurant
1,Airport West,Grocery Store,Food Truck,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Zoo Exhibit
2,Albanvale,Furniture / Home Store,Zoo Exhibit,Food Truck,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court
3,Albert Park,Athletics & Sports,Café,Breakfast Spot,Sports Club,Light Rail Station,Park,Tennis Court,Golf Course,Australian Restaurant,Playground
4,Albion,Market,Deli / Bodega,Café,Food Truck,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court
5,Alphington,Fast Food Restaurant,Farmers Market,Train Station,Liquor Store,Convenience Store,Gym / Fitness Center,Thai Restaurant,Food & Drink Shop,Fish Market,Flea Market
6,Altona,Thai Restaurant,Discount Store,Furniture / Home Store,Convenience Store,Zoo Exhibit,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court
7,Altona Meadows,Park,Dog Run,Convenience Store,Business Service,Fish & Chips Shop,Zoo Exhibit,Food Court,Flea Market,Flower Shop,Food
8,Altona North,Badminton Court,Business Service,Zoo Exhibit,Fast Food Restaurant,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court
9,Ardeer,Garden Center,Motel,Gift Shop,Football Stadium,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck


In [35]:
melbourne_grouped_clustering = melbourne_grouped.drop('Suburb', 1)

k_vals = range(2,529)
silhouette_avg = np.zeros(len(k_vals))

for i in range(0,len(k_vals)):
    # run k-means clustering
    kmeans = KMeans(n_clusters = k_vals[i], random_state = 0)
    cluster_labels = kmeans.fit_predict(melbourne_grouped_clustering)
    
    # The silhouette_score gives the average value for all the samples.
    # This gives a perspective into the density and separation of the formed
    # clusters
    silhouette_avg[i] = silhouette_score(melbourne_grouped_clustering, cluster_labels)
    print("For n_clusters =", k_vals[i], "The average silhouette_score is :", round(silhouette_avg[i],4))


For n_clusters = 2 The average silhouette_score is : 0.2627
For n_clusters = 3 The average silhouette_score is : -0.0152
For n_clusters = 4 The average silhouette_score is : 0.0021
For n_clusters = 5 The average silhouette_score is : 0.0133
For n_clusters = 6 The average silhouette_score is : 0.0266
For n_clusters = 7 The average silhouette_score is : 0.0276
For n_clusters = 8 The average silhouette_score is : 0.0408
For n_clusters = 9 The average silhouette_score is : 0.0489
For n_clusters = 10 The average silhouette_score is : 0.0342
For n_clusters = 11 The average silhouette_score is : 0.0383
For n_clusters = 12 The average silhouette_score is : 0.051
For n_clusters = 13 The average silhouette_score is : 0.0422
For n_clusters = 14 The average silhouette_score is : 0.0495
For n_clusters = 15 The average silhouette_score is : 0.0556
For n_clusters = 16 The average silhouette_score is : 0.0261
For n_clusters = 17 The average silhouette_score is : 0.0619
For n_clusters = 18 The average 

For n_clusters = 137 The average silhouette_score is : 0.1329
For n_clusters = 138 The average silhouette_score is : 0.1326
For n_clusters = 139 The average silhouette_score is : 0.1316
For n_clusters = 140 The average silhouette_score is : 0.1323
For n_clusters = 141 The average silhouette_score is : 0.1327
For n_clusters = 142 The average silhouette_score is : 0.142
For n_clusters = 143 The average silhouette_score is : 0.1264
For n_clusters = 144 The average silhouette_score is : 0.1273
For n_clusters = 145 The average silhouette_score is : 0.1282
For n_clusters = 146 The average silhouette_score is : 0.1279
For n_clusters = 147 The average silhouette_score is : 0.1283
For n_clusters = 148 The average silhouette_score is : 0.1274
For n_clusters = 149 The average silhouette_score is : 0.1371
For n_clusters = 150 The average silhouette_score is : 0.1256
For n_clusters = 151 The average silhouette_score is : 0.1326
For n_clusters = 152 The average silhouette_score is : 0.1368
For n_clu

For n_clusters = 270 The average silhouette_score is : 0.1277
For n_clusters = 271 The average silhouette_score is : 0.1275
For n_clusters = 272 The average silhouette_score is : 0.128
For n_clusters = 273 The average silhouette_score is : 0.1282
For n_clusters = 274 The average silhouette_score is : 0.1279
For n_clusters = 275 The average silhouette_score is : 0.1269
For n_clusters = 276 The average silhouette_score is : 0.1264
For n_clusters = 277 The average silhouette_score is : 0.1259
For n_clusters = 278 The average silhouette_score is : 0.1264
For n_clusters = 279 The average silhouette_score is : 0.1263
For n_clusters = 280 The average silhouette_score is : 0.1255
For n_clusters = 281 The average silhouette_score is : 0.1242
For n_clusters = 282 The average silhouette_score is : 0.1244
For n_clusters = 283 The average silhouette_score is : 0.1255
For n_clusters = 284 The average silhouette_score is : 0.1263
For n_clusters = 285 The average silhouette_score is : 0.1251
For n_clu

C:\Users\mariom1\Anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1008: ConvergenceWarning: Number of distinct clusters (345) found smaller than n_clusters (346). Possibly due to duplicate points in X.
  return self.fit(X, sample_weight=sample_weight).labels_


For n_clusters = 346 The average silhouette_score is : 0.1105


...

For n_clusters = 369 The average silhouette_score is : 0.1105


C:\Users\mariom1\Anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1008: ConvergenceWarning: Number of distinct clusters (345) found smaller than n_clusters (370). Possibly due to duplicate points in X.
  return self.fit(X, sample_weight=sample_weight).labels_


For n_clusters = 370 The average silhouette_score is : 0.1105
For n_clusters = 371 The average silhouette_score is : 0.1105


C:\Users\mariom1\Anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1008: ConvergenceWarning: Number of distinct clusters (345) found smaller than n_clusters (371). Possibly due to duplicate points in X.
  return self.fit(X, sample_weight=sample_weight).labels_


ValueError: n_samples=371 should be >= n_clusters=372